In [71]:
import json
import pandas as pd

with open("food-event-0001-of-0001.json", "r", encoding="utf-8") as file:
    data = json.load(file)

df = pd.json_normalize(data['results'])

print(df)

          report_number                                           outcomes  \
0                176698  [Hospitalization, Other Serious or Important M...   
1                177899                                    [Other Outcome]   
2                178670  [Other Serious or Important Medical Event, Vis...   
3                178913                                    [Other Outcome]   
4                179561  [Other Serious or Important Medical Event, Vis...   
...                 ...                                                ...   
140560  2023-CFS-008068  [Death, Other Serious or Important Medical Event]   
140561  2023-CFS-008297  [Other Serious or Important Medical Event, Oth...   
140562  2023-CFS-008783         [Other Serious or Important Medical Event]   
140563  2023-CFS-007705  [Death, Other Serious or Important Medical Event]   
140564  2023-CFS-008959         [Other Serious or Important Medical Event]   

       date_created                                          re

In [37]:
print(df['report_number'].isna().sum())

0


In [54]:
#Create a new column for date created to convert "yyyymmdd" into "yyyy/mm/dd"
df['date_created_format'] = df['date_created'].str[:4] + '/' + df['date_created'].str[4:6] + '/' + df['date_created'].str[6:]
# convert to datetime object
df['date_created_format'] = pd.to_datetime(df['date_created_format'], format='%Y/%m/%d')

#Create a new column for date created to convert "yyyymmdd" into "yyyy/mm/dd"
df['date_started_format'] = df['date_started'].str[:4] + '/' + df['date_started'].str[4:6] + '/' + df['date_started'].str[6:]
# convert to datetime object
df['date_started_format'] = pd.to_datetime(df['date_started_format'], format='%Y/%m/%d')

In [55]:
# delete the columns date_created and date_started

df.drop('date_created', axis=1, inplace=True)
df.drop('date_started', axis=1, inplace=True)

In [56]:
#rename the column for clarity and sql friendly query
df.rename(columns={'consumer.gender': 'consumer_gender', 'consumer.age_unit': 'consumer_age_unit', 
                   'consumer.age': 'consumer_age', 'date_created_format':'date_created', 
                   'date_started_format': 'date_started'}, inplace=True)
df.dtypes

report_number                object
outcomes                     object
reactions                    object
products                     object
consumer_age                 object
consumer_age_unit            object
consumer_gender              object
date_created         datetime64[ns]
date_started         datetime64[ns]
dtype: object

We are spotting a issue due to the nested JSON data. The column products contains nested dictionnaries.
We decided to 'flatten' this nested column and create another table so each mini-table becomes its own DataFrame.

In [57]:
# We copy the two column we will need 'report_number' and 'products'
df_products = df[['report_number', 'products']].copy()

#
df_products = df_products[df_products['products'].notna()]

#explode the list of dictionaries to match the report_number
df_products = df_products.explode('products', ignore_index=True)

df_products = df_products[df_products['products'].notna()]

#Flatten the dictionnaries
products_details = pd.json_normalize(df_products['products'])

#combine te flatten dictionnaries with the record_number
products_table = pd.concat([df_products[['report_number']], products_details], axis=1)

In [42]:
print(products_table)

          report_number     role  \
0                176698  SUSPECT   
1                177899  SUSPECT   
2                178670  SUSPECT   
3                178670  SUSPECT   
4                178670  SUSPECT   
...                 ...      ...   
221172  2023-CFS-008297  SUSPECT   
221173  2023-CFS-008783  SUSPECT   
221174  2023-CFS-007705  SUSPECT   
221175  2023-CFS-007705  SUSPECT   
221176  2023-CFS-008959  SUSPECT   

                                               name_brand industry_code  \
0                        ENDLESS YOUTH VITAMIN SUPPLEMENT            54   
1                                          AQUAFINA WATER            29   
2       OSTEO BI-FLEX WITH 5-LOXIN ADVANCED JOINT CARE...            54   
3       SPRING VALLEY BIOTIN 1000 MCG SOFTGELS (DIETAR...            54   
4       SPRING VALLEY CRANBERRY FRUIT 4200MG PLUS VITA...            54   
...                                                   ...           ...   
221172                                    

In [127]:
products_table.to_csv('products_table.csv', index=False, encoding='utf-8')

In [58]:
print(products_table['report_number'].isna().sum())

0


In [59]:
df.drop('products', axis=1, inplace=True)
print(df)

          report_number                                           outcomes  \
0                176698  [Hospitalization, Other Serious or Important M...   
1                177899                                    [Other Outcome]   
2                178670  [Other Serious or Important Medical Event, Vis...   
3                178913                                    [Other Outcome]   
4                179561  [Other Serious or Important Medical Event, Vis...   
...                 ...                                                ...   
140560  2023-CFS-008068  [Death, Other Serious or Important Medical Event]   
140561  2023-CFS-008297  [Other Serious or Important Medical Event, Oth...   
140562  2023-CFS-008783         [Other Serious or Important Medical Event]   
140563  2023-CFS-007705  [Death, Other Serious or Important Medical Event]   
140564  2023-CFS-008959         [Other Serious or Important Medical Event]   

                                                reactions consu

In [60]:
import numpy as np

#convert to numeric
df['consumer_age'] = pd.to_numeric(df['consumer_age'], errors='coerce')

#apply floor to round down
df['consumer_age'] = np.floor(df['consumer_age'])

df['consumer_age'] = df['consumer_age'].astype('Int64')

In [61]:
#remove brackets and quotes from outcomes and reactions
df['outcomes'] = df['outcomes'].astype(str).str.replace('[', '', regex=False)
df['outcomes'] = df['outcomes'].astype(str).str.replace(']', '', regex=False)
df['outcomes'] = df['outcomes'].astype(str).str.replace("'", "", regex=False)
df['reactions'] = df['reactions'].astype(str).str.replace(']', '', regex=False)
df['reactions'] = df['reactions'].astype(str).str.replace("'", "", regex=False)
df['reactions'] = df['reactions'].astype(str).str.replace('[', "", regex=False)
df['reactions'] = df['reactions'].astype(str).str.lower()

In [62]:
df.to_csv('food_event_clean.csv', index=False)

In [63]:
print(df)

          report_number                                           outcomes  \
0                176698  Hospitalization, Other Serious or Important Me...   
1                177899                                      Other Outcome   
2                178670  Other Serious or Important Medical Event, Visi...   
3                178913                                      Other Outcome   
4                179561  Other Serious or Important Medical Event, Visi...   
...                 ...                                                ...   
140560  2023-CFS-008068    Death, Other Serious or Important Medical Event   
140561  2023-CFS-008297  Other Serious or Important Medical Event, Othe...   
140562  2023-CFS-008783           Other Serious or Important Medical Event   
140563  2023-CFS-007705    Death, Other Serious or Important Medical Event   
140564  2023-CFS-008959           Other Serious or Important Medical Event   

                                                reactions  cons

In [64]:
##Convertion of all age to integer years, rounding down, based on the unit

#calculation logic
unit_to_year = {
    'year(s)': 1,
    'month(s)': 1/12,
    'week(s)': 1/52,
    'day(s)': 1/365,
    'decade(s)': 10
}

#convertion fonction
def convert_to_year(row):
    age = row['consumer_age']
    unit = row['consumer_age_unit']
    factor = unit_to_year.get(unit, np.nan)
    if pd.notnull(age) and pd.notnull(factor):
        return np.floor(age * factor)
    return np.nan

df['age_in_years'] = df.apply(convert_to_year, axis=1)

In [68]:
df['age_in_years'] = df['age_in_years'].astype('Int64')
print(df)

          report_number                                           outcomes  \
0                176698  Hospitalization, Other Serious or Important Me...   
1                177899                                      Other Outcome   
2                178670  Other Serious or Important Medical Event, Visi...   
3                178913                                      Other Outcome   
4                179561  Other Serious or Important Medical Event, Visi...   
...                 ...                                                ...   
140560  2023-CFS-008068    Death, Other Serious or Important Medical Event   
140561  2023-CFS-008297  Other Serious or Important Medical Event, Othe...   
140562  2023-CFS-008783           Other Serious or Important Medical Event   
140563  2023-CFS-007705    Death, Other Serious or Important Medical Event   
140564  2023-CFS-008959           Other Serious or Important Medical Event   

                                                reactions  cons

In [70]:
df.to_csv('food_event_clean.csv', index=False)